In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
import random
import keras
import math

from tqdm import tqdm
from scipy.stats import spearmanr
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn import preprocessing
# from keras import backend as K
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation
from lime.lime_tabular import RecurrentTabularExplainer
from methods import *

from scipy import optimize
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
from explainers import *

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [I. Mise en place des fonctions](#function)
    * [1. KernelSHAP](#KernelSHAP)
    * [2. L2X](#l2x)
    * [3. Génération des explications](#f_xai)
    * [4. Métriques d'évaluation](#metrique)
    * [5. Fonction pour la préparation des données et du modèle d'analyse](#dataprep)
* [II. Chargement des données](#data)
* [III. Génération d'explication](#xai_generation)
* [IV. Evaluation des explication](#xai_metric)

## First Bullet Header <a class="anchor" id="function"></a>

### KernelSHAP  <a id="KernelSHAP"></a>

In [2]:
class KernelSHAP:

    def __init__(self, model, nsegments=1, nsamples=1000, feature_faker=lambda _min, _max, _mean, _std, _size: 0):
        self.model = model
        self.nsegments = nsegments
        self.nsamples = nsamples
        self.feature_faker = feature_faker


    # Weight is defined in the paper https://arxiv.org/pdf/1705.07874.pdf .
    def _get_weights(self, mask_row):
        """
        Compute the weight of a sample as an measuring of the number of 1's
        """
        M = no_of_features = len(mask_row)
        z = no_of_masked_feature = np.sum(mask_row)

        weight = (M-1) /  (  comb(M,z) * z * (M-z)   )
        return weight


    def explain(self, input_array):

        mean_pred = self.model.predict(input_array)[0][0]

        if len(input_array.shape) > 2:
            input_array = input_array.squeeze()

        # compute the mean prediction
        #mean_pred = self.model.predict(np.array([mean_sample(input_array)]))[0][0]


        # create the mask to train the linear model
        nsamples = self.nsamples
        mask = []
        predictions = []
        segments = segment(input_array, self.nsegments)
        segments = sorted(segments, key=(lambda x: x[1]))
        samples = sampling(input_array, segments, n=nsamples, feature_faker=self.feature_faker)
        zs = np.array([s[1] for s in samples])
        predictions = self.model.predict(zs, batch_size=128)
        for zprime, z in samples:
            mask.append(zprime)


        # weights  of the masks
        weights = np.apply_along_axis(self._get_weights, 1, mask)

        # compute the importance coeficients
        B0 = mean_pred
        W = np.diag(weights)


        X = np.array(mask)
        y = np.array(predictions).reshape(self.nsamples,1)

        y = np.abs(y - B0)
        y = 1 - (y / y.max())

        B = np.dot(np.linalg.inv(np.dot(np.dot(X.transpose(),W),X)), np.dot(np.dot(X.transpose(),W),y))
        coef = np.copy(B.squeeze())
        
        # Essayer d'autre type de kernel non linéaire ; limite de shap l'aspect lineaire


        heatmap = np.zeros(input_array.shape, np.float32)
        for (i, s, e), imp in zip(segments, coef):
            heatmap[i,s:e] = imp

        #heatmap = cv2.resize(heatmap.T, dsize=input_array.shape, interpolation=cv2.INTER_CUBIC).T

        return heatmap


    def display(self, input_array, explanation):

        s = input_array[0].squeeze()
        aux_hm = cv2.resize(explanation.T, dsize=s.shape, interpolation=cv2.INTER_CUBIC).T

        aux_hm =  (aux_hm - aux_hm.min()) / (aux_hm.max() - aux_hm.min())
        importances = [[(i, j, j+1, aux_hm[i,j]) for j in range(s.shape[1])] for i in range(s.shape[0])]

        display(s.T, importances)



def validate(explainer, sample, iterations=100, nstd=1.5, top_features=100):
    """
    N points are chosen from those with the highest score and are perturbed to create a new series (tp). 
    Another series (tr) is also created by adding noise to another n random points. 
    The importance qm of points must comply with the following rule qm(t) >= qm(tr) > qm(tp)
    """

    base_exp = explainer.explain(sample)
    if len(sample.shape) > 3:
        sample = sample[0,:,:,0]
    else:
        sample = sample[0]
    shape = sample.shape

    _mean, _std = base_exp.mean(), base_exp.std()
    theshold = _mean + nstd*_std
    nsamples = (base_exp > theshold).sum()
    nsamples = min(nsamples, top_features)
    aux = base_exp.flatten()
    theshold = aux[aux.argsort()][-nsamples]
    top_mask = aux >= theshold

    aux1 = aux


    # tc
    tc = np.copy(sample)
    assert (base_exp >= theshold).any()
    tc[base_exp >= theshold] = 0
    tc_exp = explainer.explain(tc.reshape((1, shape[0], shape[1], 1)))

    aux = tc_exp.flatten()
    theshold = aux[aux.argsort()][-nsamples]
    aux_max = aux.argsort().max()

    aux2 = aux

    # ranking
    top_tc_exp2 = (1- (aux.argsort()[top_mask] / aux_max)).mean()

    # number of noised samples outside top
    top_tc_exp1 = (aux < theshold)[top_mask].mean()


    # tr
    s = base_exp.shape[0] * base_exp.shape[1]
    t_mask = np.zeros(s, dtype=bool)
    t_mask[np.random.choice(s, nsamples, replace=False)] = True
    t_mask = t_mask.reshape(base_exp.shape)
    tr = np.copy(sample)
    tr[t_mask] = 0
    tr_exp = explainer.explain(tr.reshape((1, shape[0], shape[1], 1)))

    #
    base_exp = base_exp / base_exp.max()
    tc_exp = base_exp / tc_exp.max()


    return (nsamples, top_tc_exp1, top_tc_exp2)

### L2X  <a id="l2x"></a>

In [3]:
class L2X:
    """
    Rhe original l2x out put a vector of binary feature importance values for a
    chosen k, with k being the number of important features the user thinks is in
    the dataset. First of all user has to choose it, it's unclear how you would choose
    the best k. Second, since all important values have weights of 1, it is
    impossible to rank them.

    To overcome this, Yang proposes to run l2x for k = 1,2,...,M and add feature
    importance values. This way the most important feature will have a final importance
    value of M, because it will be 1 in each run.

    """

    def __init__(self, f, X, **kwargs):
        self.f = f
        self.X = X #self.X = X.values
        x_reshaped = X.reshape((X.shape[0], -1)) #*****************
        self.x_reshaped = x_reshaped
        self.M = x_reshaped.shape[1]
        if 'batch_size' in kwargs:
            BATCH_SIZE = kwargs['batch_size']
        # set up models with k = 1,2,3,..., M
        self.models = []
        self.pred_models = []
        self.Y = self.f(X)
        # print("X shape", X.shape)
        for k in range(1, 100):
            model, pred_model = buildmodel(self.x_reshaped, self.Y, k, self.M)
            self.models.append(model)
            self.pred_models.append(pred_model)

    def explain(self, x):
        weights = np.zeros_like(x)
        #x = np.ones_like(x)
        self.expected_values = np.ones((x.shape[0], 1)) * np.mean(self.Y)
        for i in range(len(self.models)):
            # if i == 3:
                weights = weights + self.pred_models[i].predict(
                    x, verbose=False, batch_size=BATCH_SIZE
                )
        # normalize
        weights = weights / np.expand_dims(np.sum(weights, axis=1), 1)
                # print('k:', i+1)
        #print(weights[:10])
                # print(np.sum(weights,axis=0))
                # median_ranks = compute_median_rank(weights,4)
                # print(np.mean(median_ranks))
        return weights

### Fonction pour la génération des explications <a id="f_xai"></a>

In [4]:
# define fuction to get all explanations data test

# Function explanation for lime
def get_lime_explanation(data, e, L2X=False) :
    # e  = fn = model.predict
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(data.shape[1]*data.shape[2])])

    # Get explanations
    for row in range(data.shape[0]) : 
        explanation = lime_explainer.explain_instance(data[row],
                                                      e,
                                                      num_features=data.shape[1]*data.shape[2]) 
        # fn = model.predict, initialize lime_explainer = Reccurent()
        lime_values = explanation.local_exp[1]
        # Add explanation in df_explanation
        lime_dict = {}
        for tup in lime_values :
            lime_dict[str(tup[0])] = tup[1]
        df_expplanation.loc[len(df_expplanation)] = lime_dict
    
    return df_expplanation

# # Function explanation for others
def get_explainations(data, e, L2X = False) :
    
    # df diemnsion
    if L2X==True :
        X_to_def_col = data[0:1]
        explanation_test = e.explain(X_to_def_col.reshape((X_to_def_col.shape[0], -1)))
        num_columns = explanation_test.flatten().shape[0]
        
    else : 
        explanation_test = e.explain(data[0:1])
        num_columns = explanation_test.flatten().shape[0]
    
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(num_columns)])

    # Get explanations
    for row in range(data.shape[0]) :
        if L2X==True:
            X_row = data[row:row+1]
            explanation = e.explain(X_row.reshape((X_row.shape[0], -1)))
        else :
            explanation = e.explain(data[row:row+1])
        # Add explanation in df_explanation
        explanation = explanation.flatten()
        feature_dict = {}
        for i in range(num_columns) :
            feature_dict[str(i)] = explanation[i]
        df_expplanation.loc[len(df_expplanation)] = feature_dict
    
    return df_expplanation

# run metrics and compare

### Métriques d'évaluation <a id="metrique"></a>

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  9 10:55:10 2023

@author: mlndao
"""

from tqdm import tqdm

def identity(X_dist, E_dist) : 
    
    '''
        The principle of identity states that identical objects should receive identical explanations. This is 
        a measure of the level of intrinsic non-determinism in the method:
        
                                    d(xa , xb ) = 0 => ∀a,b d(εa , εb ) = 0
    ''' 

    # Defining a function to navigate around the distance matrix
    i_dm = X_dist.values #Computations on np.arrays are faster than on dataframes
    l_e_dm = E_dist.values
    errors = []
    
    # To run the loop with a progress bar
    for column in X_dist:
        for row in X_dist:
            if i_dm[row, column] == 0: #If two inputs have distance 0, then their explanations must too
                if l_e_dm[row, column] == 0:
                    errors.append(1)
                else:
                    errors.append(0)
        
        return np.nanmean(errors)

def separability(X_dist, E_dist):
    """
     Non-identical objects can not have identical explanations:
    
                 ∀a,b; d(xa , xb ) ̸= 0 =>  d(εa , εb ) > 0

     This proxy is based on the assumption that every feature has a minimum level of importance,
     positive or negative, in the predictions. The idea is that if a feature is not actually 
     needed for the prediction, then two samples that differ only in that feature will 
     have the same prediction. In this scenario, the explanation method could provide the same 
     explanation, even though the samples are different.

    """
    
    i_dm = X_dist.values #Computations on np.arrays are faster than on dataframes
    l_e_dm = E_dist.values
    errors = []
    
    # To run the loop with a progress bar
    for column in X_dist:
        for row in X_dist:
            if i_dm[row, column] > 0: #If two inputs have distance 0, then their explanations must too
                if l_e_dm[row, column] > 0:
                    errors.append(1)
                else:
                    errors.append(0)
        
        return np.nanmean(errors)

def stability(X_dist, E_dist):
    """
     Non-identical objects can not have identical explanations:
    
                 ∀a,b; d(xa , xb ) ̸= 0 =>  d(εa , εb ) > 0

     This proxy is based on the assumption that every feature has a minimum level of importance,
     positive or negative, in the predictions. The idea is that if a feature is not actually 
     needed for the prediction, then two samples that differ only in that feature will 
     have the same prediction. In this scenario, the explanation method could provide the same 
     explanation, even though the samples are different.

    """
    
    errors = []
    
    #Creating a list that contains all Spearman's Rhos rank correlation coefficients
    sp_rhos = list()
    
    #Creating a function that computes all Rhos
    for column in X_dist:
        sp_rho = spearmanr(X_dist.iloc[:,column], E_dist.iloc[:,column])
        sp_rhos.append(sp_rho[0])
    
    # Defining a function to navigate around the distance matrix

    for entry in sp_rhos:
        if entry >= 0:
            errors.append(1)
        else:
            errors.append(0)
    
    return np.nanmean(errors)

#%% 

def coherence(model, explainer, samples, targets, e, nstd=1.5, top_features=5, verbose=True, L2X = False):
    '''
    model : model.predic
    explainer : get_explanation's function
    sample : data
    target : label
    e : e like KenrelSHAP
    '''
    explains = []
    valid_idx = []
    # for i in tqdm.tqdm(range(len(samples)), total=len(samples), disable=not verbose):
    for i in range(len(samples)):
        xi = samples[i:i+1] 
        exp = explainer(xi, e, L2X).values
        # exp = explainer(xi, lime_explainer, predict_with_xgboost).values

        # compute the thresold using mean + n*std 
        _mean, _std = exp.mean(), exp.std()
        theshold = _mean + nstd*_std
        nsamples = (exp > theshold).sum()
        nsamples = min(nsamples, top_features)
        aux = exp.flatten()
        theshold = aux[aux.argsort()][-nsamples]
        indexes = np.argwhere(exp.flatten() < theshold)

        # remove that features
        exp[exp < theshold] = 0
        xic = np.copy(xi).flatten()
        xic[indexes] = 0
        xic = xic.reshape(xi.shape)

        if not np.isnan(exp).any():
            valid_idx.append(i)
            explains.append(xic)

    samples = samples[valid_idx]
    targets = targets[valid_idx]
    # dtest = model.DMatrix(samples, targets)


    tmax = targets.max()
    targets = targets / tmax

    pred = model(samples) / tmax
    pred = pred.reshape(targets.shape)
#     errors = 1 - (pred - targets) ** 2
    errors = (pred - targets) ** 2 # modified

    exp = np.array(explains).reshape(samples.shape)

    explains = np.array(explains).reshape(samples.shape)
    exp_pred = model(explains) / tmax
#     exp_errors = 1- (exp_pred - targets) ** 2
    exp_errors = (exp_pred - targets) ** 2 #modified
    
    coherence_i = np.abs(errors - exp_errors)
    coherence = np.mean(coherence_i)
    completeness = min(np.mean(exp_errors / errors), np.mean(errors / exp_errors))
    congruency = np.sqrt(np.mean((coherence_i - coherence)**2))
    
#     return {
#             'coherence': coherence, 
#             'completeness':np.mean(exp_errors / errors),
#             'congruency': np.sqrt(np.mean((coherence_i - coherence)**2))
#            }
    return coherence, completeness, congruency

#%%

def selectivity(model, explainer, samples, e_x, L2X = False, samples_chunk=1, verbose=True):
    """
    The elimination of relevant variables must affect 
    negatively to the prediction. To compute the selectivity 
    the features are ordered from most to lest relevant. 
    One by one the features are removed, set to zero for 
    example, and the residual errors are obtained to get the 
    area under the curve (AUC).
    
    - model : model.predic
    - explainer : get_explanation's function
    - sample : data
    - e_x : e like KenrelSHAP
    """

    errors = []
    # for i in tqdm.tqdm(range(len(samples)-1), total=len(samples), disable=not verbose):
    for i in tqdm(range(len(samples))):
        dxs, des = [], []
        xi = samples[i:i+1]
        ei = explainer(xi, e_x, L2X).values
        if np.isnan(ei).any():
            continue
            
        idxs = ei.flatten().argsort()[::-1]
        xi = xi[0]
        xs = [xi]
        xprime = xi.flatten()
        l = idxs.shape[0]
        if samples_chunk >= 1:
            idxs = np.split(idxs, int(l/samples_chunk))
        
        for i in idxs:
            xprime[i] = 0
            xs.append(xprime.reshape(xi.shape))
            xprime = np.copy(xprime)
            
        preds = model(np.array(xs))   
        e = np.abs(preds[1:] - preds[:-1]) / np.abs(preds[0] + 1e-12)
        e = np.cumsum(e)
        e = 1 - (e / (e.max() + 1e-12))
        score = 1 - np.mean(e)
        
        errors.append(score)
#         print('ok')
        
    return np.nanmean(errors)

#%%

def acumen(explainer, samples, e, L2X=False, iterations=100, nstd=1.5, top_features=5, verbose=True):
    """
    Se eleccionan n puntos de los que tienen mayor score y se perturban creando una nueva
    serie (tp). También se crea otra serie (tr) metiendo ruido en otros n puntos aleatorios. 
    La importancia qm de puntos debe cumplir las siguiente regla qm(t) >= qm(tr) > qm(tp)
    
    - model : model.predic
    - explainer : get_explanation's function
    - sample : data
    - e : e like KenrelSHAP
    """
    
    ranking = []
    # for i in tqdm.tqdm(range(len(samples)), total=len(samples), disable=not verbose):
    for i in tqdm(range(len(samples))):

        xi = samples[i:i+1] 
        #pred
        base_exp = explainer(xi, e, L2X).values

        if not np.isnan(base_exp).any():
            # compute the thresold using mean + n*std 
            _mean, _std = base_exp.mean(), base_exp.std()
            theshold = _mean + nstd*_std
            nsamples = (base_exp > theshold).sum()
            nsamples = min(nsamples, top_features)
            aux1 = base_exp.flatten()

            top_mask = aux1.argsort()[-nsamples:]

            # tc
            
            tc = np.copy(xi).reshape(base_exp.shape)
            tc[base_exp >= theshold] = 0
            tc_exp = explainer(tc.reshape(xi.shape), e, L2X).values
            # pred_

            # I obtain the n most important samples of x'.
            aux = tc_exp.flatten()
            aux_max = aux.argsort().max()

            # ranking
            ranking.append((1- (np.argsort(aux).argsort()[top_mask] / aux_max)).mean())
        
    return np.nanmean(ranking)

def stability_Velmurugan(explainer, samples, e, top_features=5, verbose=True, L2X = False, nb_iter = 10):
    
    idx_ind = [i for i in range(samples.shape[0])]
    feat_list = [i for i in range(560)] # tw*nb_feature à rectifier
    list_stab_values = []
    
    for ind in tqdm(idx_ind):
        list_iter = [sample[ind:ind+1] for i in range(nb_iter)]
        Z = []
        for d in list_iter : 
            ex = explainer(d, e = e, L2X=L2X) # Get explanation

            exp_abs = np.abs(ex) # get Zi
            each = exp_abs 
            weighted = list(each.values.flatten())
            nb_feature = 10
            weighted.sort()
            max_feat = weighted[-nb_feature]

            Zi = [0]*len(feat_list)
            for i, w in enumerate(each.values.flatten()) : 
                if w>=max_feat:
                    Zi[i] = 1
            print(sum(Zi))
            Z.append(Zi)
        stab_value = st.getStability(Z) # Calculate Stability
        list_stab_values.append(stab_value)
        
    return np.mean(list_stab_values)

In [6]:
def fidelity(explainer, samples, e, top_features=5, verbose=True, L2X = False, nb_iter = 10):
    pass

def instability(explainer, samples, e, top_features=5, verbose=True, L2X = False, nb_iter = 10):
    pass

def consistency(explainer, samples, e, top_features=5, verbose=True, L2X = False, nb_iter = 10):
    pass

def sparsity(explainer, samples, e, top_features=5, verbose=True, L2X = False, nb_iter = 10):
    pass

def faithfullness():
    pass

### Fonction pour la préparation des données et du modèle d'analyse  <a id = "dataprep"></a>

In [7]:
def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
#     train['RUL'].clip(upper=upper, inplace=True)
    return X_train_interim, X_test_interim

learning_rate_ = 0.001
# input_shape = (sequence_length, train_array.shape[2])
def create_lstm_1layer(dropout, activation, weights_file, input_shape):
    history = tf.keras.callbacks.History()
    model = Sequential()
    model.add(LSTM(units=64, activation='tanh',input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='relu'))
    model.compile(loss='mse',metrics=['mse'], optimizer= 'adam') # A voir si varier ou pas s-score, rmse
    model.save_weights(weights_file)
    
    return model

def create_lstm_2layers(dropout, activation, weights_file, input_shape):
    history = tf.keras.callbacks.History()
    model = Sequential()
    model.add(LSTM(units=64, activation='tanh',return_sequences=True,input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(units=32, activation='tanh'))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='relu'))
    model.compile(loss='mse',metrics=['mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
#     model.save_weights(weights_file) 
    return model 

def create_lstm_3layers(dropout, activation, weights_file, input_shape):
#     history = tf.keras.callbacks.History()
    model = Sequential()
    model.add(LSTM(units=64, activation='tanh',return_sequences=True,input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(units=32, activation='tanh')) # A reduire et pourquoi
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mse',metrics=['mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    model.save_weights(weights_file)
    return model 

def create_lstm_4layers(dropout, activation, weights_file, input_shape):
#     history = tf.keras.callbacks.History()
    model = Sequential()
    model.add(LSTM(units=64, activation='tanh',return_sequences=True,input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(units=32, activation='tanh')) # A reduire et pourquoi
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mse',metrics=['mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    model.save_weights(weights_file)
    
    return model

def create_lstm_5layers(dropout, activation, weights_file, input_shape):
#     history = tf.keras.callbacks.History()
    model = Sequential()
    model.add(LSTM(units=64, activation='tanh',return_sequences=True,input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(LSTM(units=32, activation='tanh')) # A reduire et pourquoi
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=32, activation='selu'))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='mse',metrics=['mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    model.save_weights(weights_file)
    
    return model

def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [I. Mise en place des fonctions](#function)
    * [1. KernelSHAP](#KernelSHAP)
    * [2. L2X](#l2x)
    * [3. Génération des explications](#f_xai)
    * [4. Métriques d'évaluation](#metrique)
    * [5. Fonction pour la préparation des données et du modèle d'analyse](#dataprep)
* [II. Chargement des données](#data)
* [III. Expérimentation](#experiementation)
    * [1. Model à une couche](#model1layer)
    * [2. Model à 2 couches](#model2layer)
    * [3. Model à 3 couches](#model3layer)
    * [4. Model à 4 couches](#model4layer)
    * [5. Model à 5 couches](#model5layer)
* [IV. Evaluation des explication](#xai_metric)

### Chargement des données  <a id = "data"></a>

In [12]:
# Data preparation
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(61249, 27) (41214, 26) (248, 1)


## Expérimentations <a class="anchor" id="experiementation"></a>

### Model à une couche <a id="model1layer"></a>

In [11]:
# Modeling
if True:
    
#     print(elm)
    print('...')
    sequence_length = 40
    alpha = 0.5
    upper = 100
    learning_rate_ = 0.001
    dropout = 0.2
    activation = 'tanh'
    epochs = 20
    batch_size = 128
    train_recrul = rul_piecewise_fct(train, upper)
    weights_file = str(alpha) + '1lstm_hyper_parameter_weights.h5'
    input_shape = (sequence_length, len(remaining_sensors))
#     model = create_lstm_1layer(dropout, activation, weights_file, input_shape=input_shape)
    mse = []

    X_train_interim = add_operating_condition(train_recrul)
    X_test_interim = add_operating_condition(test)

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha=alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha=alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    print(train_array.shape, label_array.shape, test_array.shape)

    # create train-val split
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=42)
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    test_rul = rul_piecewise_fct(y_test,upper)
    
#     with tf.device('/device:GPU:0'):
#         for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
#             train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
#             train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
#             train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

#             val_unit = X_train_interim['Unit'].unique()[val_unit]
#             val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
#             val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

#             # train and evaluate model
#             model.compile(loss='mse', optimizer='adam')
# #             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

# #             with tf.device('/device:GPU:0'):
#             history = model.fit(train_split_array, train_split_label,
#                                 validation_data=(val_split_array, val_split_label),
#                                 epochs=epochs,
#                                 batch_size=batch_size,
#                                 verbose=0)
# #             mse.append(history.history['val_loss'][-1])
#             test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
#                             for unit_nr in X_test_interim['Unit'].unique())
#             test_array = np.concatenate(list(test_gen)).astype(np.float32)
#             mse_val.append(history.history['val_loss'][-1])
#             y_hat_val_split = model.predict(test_array)
#             R2_val.append(r2_score(test_rul, y_hat_val_split))
#             RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
#             score_val.append(compute_s_score(test_rul, y_hat_val_split))
#     d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
#                 'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}

...
(51538, 40, 14) (51538, 1) (248, 40, 14)


In [22]:
output = []
output.append(d)
model_1c = model
output

[{'RMSE_val': array([12.55781551,  9.87828956,  9.4201687 ]),
  'RMSE_test': [12.859532985813635, 12.095526710549679, 13.587672280687546],
  'R2_test': 0.8629494930038405,
  'Score_test': 851.3852822448085,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40}]

In [70]:
model_test = tf.keras.models.load_model('/sfds_result/model_2c')

In [71]:
model_test.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 40, 64)            20224     
                                                                 
 dropout_3 (Dropout)         (None, 40, 64)            0         
                                                                 
 lstm_4 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_101 (Dense)           (None, 1)                 33        
                                                                 
Total params: 32673 (127.63 KB)
Trainable params: 32673 (127.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model = tf.keras.models.load_model('/sfds_result/model_2c')

In [15]:
# model = best_model
train_array_1c = train_array
test_array_1c = test_array
label_array_1c = label_array

print(" Model evaluation for FD004 : ")
y_hat_train = model.predict(train_array)
evaluate(label_array, y_hat_train, 'train')

y_hat_test = model.predict(test_array)
evaluate(y_test, y_hat_test)
print(' ')
print(" Model evaluation for FD004 : ")

y_hat_test = model.predict(test_array)
evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
print(' ')
print("S-score for test : ",compute_s_score(y_test, y_hat_test))
print(' ')

 Model evaluation for FD004 : 
1611/1611 [==============================] - 22s 13ms/step
train set RMSE:9.073166847229004, R2:0.9208064608032496
8/8 [==============================] - 0s 12ms/step
test set RMSE:9.888630401663995, R2:0.918990677992583
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 13ms/step
test set RMSE:9.888630401663995, R2:0.918990677992583
 
S-score for test :  445.6088233055366
 


In [23]:
# # model = best_model
# train_array_1c = train_array
# test_array_1c = test_array
# label_array_1c = label_array

# print(" Model evaluation for FD004 : ")
# y_hat_train = model.predict(train_array)
# evaluate(label_array, y_hat_train, 'train')

# y_hat_test = model.predict(test_array)
# evaluate(y_test, y_hat_test)
# print(' ')
# print(" Model evaluation for FD004 : ")

# y_hat_test = model.predict(test_array)
# evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
# print(' ')
# print("S-score for test : ",compute_s_score(y_test, y_hat_test))
# print(' ')

 Model evaluation for FD004 : 
1611/1611 [==============================] - 14s 9ms/step
train set RMSE:9.783175468444824, R2:0.9079271538464585
8/8 [==============================] - 0s 7ms/step
test set RMSE:13.587672280687546, R2:0.8470488189754622
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 7ms/step
test set RMSE:13.587672280687546, R2:0.8470488189754622
 
S-score for test :  832.8036687850811
 


In [84]:
RMSE = [13.58]
Sscore = [832.8]

In [78]:
model = model_4c
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 40, 64)            20224     
                                                                 
 dropout_8 (Dropout)         (None, 40, 64)            0         
                                                                 
 lstm_8 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_302 (Dense)           (None, 32)                1056      
                                                                 
 dropout_10 (Dropout)        (None, 32)                0         
                                                                 
 dense_303 (Dense)           (None, 32)               

### One layer Explanation <a id="exp1"></a>

In [79]:
# df_metrics5 = pd.DataFrame()
# model = model_2c
for rd in range(5):
    # Echantillonage
    n_individus = test_array_1c.shape[0]

    # # Choisir aléatoirement 10 indices d'individus
    np.random.seed(rd)
    indices_choisis = np.random.choice(n_individus, size=10, replace=False)

    # # Sélectionner les données correspondant aux indices choisis
    test_array_sampling = test_array_1c[indices_choisis, :, :]
    label_array_sampling = y_test.values[indices_choisis, :]

    # # Afficher les dimensions des données sélectionnées
    print(test_array_sampling.shape, label_array_sampling.shape)
    # distance matrix XX'
    X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

    if True :
        # LIME
        lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                           feature_names=remaining_sensors,
                                                           mode = 'regression',
                                                           )
        lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
        Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

        #Lime's metrics
        list_metrics_lime = {}
        list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
        list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
        list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
        list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = model.predict)
        list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                   samples=test_array_sampling, e_x = model.predict)
        list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
        list_metrics_lime['explainer'] = 'lime_4c'
        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_lime])])
        print("Lime OK!!")

        # SHAP
        e = KernelSHAP(model)
        shapvalues = get_explainations(test_array_sampling, e)
        shapvalues.shape

        list_metrics_shap = {}
        shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix

        list_metrics_shap['identity'] = identity(X_dist, shap_dist)
        list_metrics_shap['separability'] = separability(X_dist, shap_dist)
        list_metrics_shap['stability'] = stability(X_dist, shap_dist)
        list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_explainations,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = e)
        list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e)
        list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)

        list_metrics_shap['explainer'] = 'shap_4c'
        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_shap])])
        print("shap OK !!")

        # L2X
        e = L2X(model.predict, test_array_sampling)
        l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
        l2xvalues.shape

        # l2x's metrics
        list_metrics_l2x = {}
        l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

        list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
        list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
        list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
        list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                   samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
        list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e, L2X=True)
        list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
        list_metrics_l2x['explainer'] = 'l2x_4c'
        print("L2X OK !!")

        df_metrics5 = pd.concat([df_metrics5, pd.DataFrame([list_metrics_l2x])])
df_metrics5

(10, 40, 14) (10, 1)
1/1 [==============================] - 0s 63ms/step



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

18/18 [==============================] - 1s 72ms/step



 10%|████████▎                                                                          | 1/10 [00:22<03:26, 22.99s/it]

18/18 [==============================] - 1s 70ms/step


...


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:06<00:14, 14.17s/it]

18/18 [==============================] - 1s 64ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:14<00:00, 25.45s/it]

L2X OK !!


,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,explainer
0,1.0,1.0,1.0,0.250002,123.782043,0.306072,0.614124,0.042397,lime_1c
0,1.0,1.0,1.0,0.147780,0.473728,0.183380,0.568432,0.313524,shap_1c
0,1.0,1.0,0.9,0.181704,2.019102,0.222921,0.620089,0.035886,l2x_1c
0,1.0,1.0,1.0,0.161335,573.330444,0.185664,0.555408,0.119141,lime_1c
0,1.0,1.0,1.0,0.157563,0.426702,0.181761,0.593014,0.472039,shap_1c
0,1.0,1.0,1.0,0.155116,0.608091,0.179124,0.544497,0.052487,l2x_1c
0,1.0,1.0,1.0,0.144818,32.490021,0.245558,0.630698,0.044520,lime_1c
0,1.0,1.0,0.9,0.141376,0.547619,0.243418,0.660767,0.583059,shap_1c
0,1.0,1.0,1.0,0.174406,0.205540,0.271803,0.669541,0.035313,l2x_1c
0,1.0,1.0,1.0,0.227306,0.198970,0.251178,0.568998,0.113453,lime_1c


In [80]:
df_metrics5.to_csv('sfds_result/df_metrics5_sfds.csv')

In [37]:
# model_1c = model
model_1c.save('/sfds_result/model_1c')

INFO:tensorflow:Assets written to: /sfds_result/model_1c\assets


INFO:tensorflow:Assets written to: /sfds_result/model_1c\assets


## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [I. Mise en place des fonctions](#function)
    * [1. KernelSHAP](#KernelSHAP)
    * [2. L2X](#l2x)
    * [3. Génération des explications](#f_xai)
    * [4. Métriques d'évaluation](#metrique)
    * [5. Fonction pour la préparation des données et du modèle d'analyse](#dataprep)
* [II. Chargement des données](#data)
* [III. Expérimentation](#experiementation)
    * [1. Model à une couche](#model1layer)
    * [2. Model à deux couches](#model2layer)
    * [3. Model à 3 couches](#model3layer)
    * [4. Model à 4 couches](#model4layer)
    * [5. Model à 5 couches](#model5layer)
* [IV. Evaluation des explication](#xai_metric)

### Model à 2 couches <a id="model2layer"></a>

In [33]:
# Modeling
if True:
    weights_file = str(alpha) + '2lstm_hyper_parameter_weights.h5'
    model = create_lstm_2layers(dropout, activation, weights_file, input_shape=input_shape)
    mse = []
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer='adam')
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

#             with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}
output.append(d)
model_2c = model
output

8/8 [==============================] - 1s 12ms/step


[{'RMSE_val': array([12.55781551,  9.87828956,  9.4201687 ]),
  'RMSE_test': [12.859532985813635, 12.095526710549679, 13.587672280687546],
  'R2_test': 0.8629494930038405,
  'Score_test': 851.3852822448085,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40},
 {'RMSE_val': array([32.47388148,  9.54485333,  8.73792065]),
  'RMSE_test': [35.24477417004867, 13.600942821270046, 9.888630401663995],
  'R2_test': 0.5788844663564608,
  'Score_test': 10162.63439248012,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40}]

In [34]:
print(" Model evaluation for FD004 : ")
y_hat_train = model.predict(train_array)
evaluate(label_array, y_hat_train, 'train')

y_hat_test = model.predict(test_array)
evaluate(y_test, y_hat_test)
print(' ')
print(" Model evaluation for FD004 : ")

y_hat_test = model.predict(test_array)
evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
print(' ')
print("S-score for test : ",compute_s_score(y_test, y_hat_test))
print(' ')

 Model evaluation for FD004 : 
1611/1611 [==============================] - 27s 17ms/step
train set RMSE:9.073166847229004, R2:0.9208064608032496
8/8 [==============================] - 0s 14ms/step
test set RMSE:9.888630401663995, R2:0.918990677992583
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 15ms/step
test set RMSE:9.888630401663995, R2:0.918990677992583
 
S-score for test :  445.6088233055366
 


In [85]:
RMSE.append(9.88)
Sscore.append(445.61)

### 2 Layers explanation

In [35]:
if True:
    
    # LIME
    lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                       feature_names=remaining_sensors,
                                                       mode = 'regression',
                                                       )
    lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
    Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

    #Lime's metrics
    list_metrics_lime = {}
    list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
    list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
    list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
    list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                explainer = get_lime_explanation,
                                               samples=test_array_sampling,
                                                targets=label_array, e = model.predict)
    list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                               samples=test_array_sampling, e_x = model.predict)
    list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
    list_metrics_lime['explainer'] = 'lime_1c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])
    print("Lime OK!!")
    
    # SHAP
    e = KernelSHAP(model)
    shapvalues = get_explainations(test_array_sampling, e)
    shapvalues.shape

    list_metrics_shap = {}
    shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix
    
    list_metrics_shap['identity'] = identity(X_dist, shap_dist)
    list_metrics_shap['separability'] = separability(X_dist, shap_dist)
    list_metrics_shap['stability'] = stability(X_dist, shap_dist)
    list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                explainer = get_explainations,
                                               samples=test_array_sampling,
                                                targets=label_array, e = e)
    list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e)
    list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)

    list_metrics_shap['explainer'] = 'shap_1c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
    print("shap OK !!")
    
    # L2X
    e = L2X(model.predict, test_array_sampling)
    l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
    l2xvalues.shape

    # l2x's metrics
    list_metrics_l2x = {}
    l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

    list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
    list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
    list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
    list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                               samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
    list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e, L2X=True)
    list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
    list_metrics_l2x['explainer'] = 'l2x_1c'
    print("L2X OK !!")

    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])
df_metrics

1/1 [==============================] - 0s 51ms/step



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

18/18 [==============================] - 0s 15ms/step



  5%|████▏                                                                              | 1/20 [00:04<01:18,  4.16s/it]

18/18 [==============================] - 0s 19ms/step


...


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:09<00:12, 12.20s/it]

18/18 [==============================] - 1s 33ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:32<00:00, 25.63s/it]

L2X OK !!


,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,explainer
0,1.0,1.0,1.00,0.150924,4.145610,0.236745,0.631832,0.048846,lime_1c
0,1.0,1.0,0.95,0.147892,0.568901,0.232912,0.587377,0.453238,shap_1c
0,1.0,1.0,1.00,0.216381,2.378321,0.308456,0.615225,0.034186,l2x_1c
0,1.0,1.0,0.95,0.160015,0.218664,0.250107,0.744438,0.108766,lime_1c
0,1.0,1.0,0.95,0.157933,0.333230,0.251317,0.728482,0.434705,shap_1c
0,1.0,1.0,0.90,0.176160,38.070158,0.262740,0.783042,0.029785,l2x_1c


In [48]:
# model_2c = model
model_2c.save('/sfds_result/model_2c')

INFO:tensorflow:Assets written to: /sfds_result/model_2c\assets


INFO:tensorflow:Assets written to: /sfds_result/model_2c\assets


In [45]:
df_metrics.to_csv('sfds_result/df_metric_sfds.csv')

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [I. Mise en place des fonctions](#function)
    * [1. KernelSHAP](#KernelSHAP)
    * [2. L2X](#l2x)
    * [3. Génération des explications](#f_xai)
    * [4. Métriques d'évaluation](#metrique)
    * [5. Fonction pour la préparation des données et du modèle d'analyse](#dataprep)
* [II. Chargement des données](#data)
* [III. Expérimentation](#experiementation)
    * [1. Model à une couche](#model1layer)
    * [2. Model à 2 couches](#model2layer)
    * [3. Model à 3 couches](#model3layer)
    * [4. Model à 4 couches](#model4layer)
    * [5. Model à 5 couches](#model5layer)
* [IV. Evaluation des explication](#xai_metric)

### Model à 3 couches <a id="model3layer"></a>

In [49]:
# Modeling
if True:
    weights_file = str(alpha) + '2lstm_hyper_parameter_weights.h5'
    model = create_lstm_3layers(dropout, activation, weights_file, input_shape=input_shape)
    mse = []
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer='adam')
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

#             with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}
output.append(d)
model_3c = model
output

8/8 [==============================] - 1s 24ms/step


[{'RMSE_val': array([12.55781551,  9.87828956,  9.4201687 ]),
  'RMSE_test': [12.859532985813635, 12.095526710549679, 13.587672280687546],
  'R2_test': 0.8629494930038405,
  'Score_test': 851.3852822448085,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40},
 {'RMSE_val': array([32.47388148,  9.54485333,  8.73792065]),
  'RMSE_test': [35.24477417004867, 13.600942821270046, 9.888630401663995],
  'R2_test': 0.5788844663564608,
  'Score_test': 10162.63439248012,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40},
 {'RMSE_val': array([12.1541831 ,  8.68845324,  8.90311545]),
  'RMSE_test': [10.523411858294384, 9.951691682882839, 10.563311250175824],
  'R2_test': 0.9112566433687842,
  'Score_test': 456.65256172335603,
  'alpha': 0.5,
  'rul_piecewise': 100,
  'sequence_length': 40}]

In [50]:
print(" Model evaluation for FD004 : ")
y_hat_train = model.predict(train_array)
evaluate(label_array, y_hat_train, 'train')

y_hat_test = model.predict(test_array)
evaluate(y_test, y_hat_test)
print(' ')
print(" Model evaluation for FD004 : ")

y_hat_test = model.predict(test_array)
evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
print(' ')
print("S-score for test : ",compute_s_score(y_test, y_hat_test))
print(' ')

 Model evaluation for FD004 : 
1611/1611 [==============================] - 62s 38ms/step
train set RMSE:8.85003662109375, R2:0.9246536818814984
8/8 [==============================] - 0s 49ms/step
test set RMSE:10.563311250175824, R2:0.9075593776612392
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 43ms/step
test set RMSE:10.563311250175824, R2:0.9075593776612392
 
S-score for test :  479.67525622443407
 


In [86]:
RMSE.append(10.56)
Sscore.append(479.67)

### 3 Layers explanation

In [51]:
if True:
    
    # LIME
    lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                       feature_names=remaining_sensors,
                                                       mode = 'regression',
                                                       )
    lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
    Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

    #Lime's metrics
    list_metrics_lime = {}
    list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
    list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
    list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
    list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                explainer = get_lime_explanation,
                                               samples=test_array_sampling,
                                                targets=label_array, e = model.predict)
    list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                               samples=test_array_sampling, e_x = model.predict)
    list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
    list_metrics_lime['explainer'] = 'lime_1c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])
    print("Lime OK!!")
    
    # SHAP
    e = KernelSHAP(model)
    shapvalues = get_explainations(test_array_sampling, e)
    shapvalues.shape

    list_metrics_shap = {}
    shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix
    
    list_metrics_shap['identity'] = identity(X_dist, shap_dist)
    list_metrics_shap['separability'] = separability(X_dist, shap_dist)
    list_metrics_shap['stability'] = stability(X_dist, shap_dist)
    list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                explainer = get_explainations,
                                               samples=test_array_sampling,
                                                targets=label_array, e = e)
    list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e)
    list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)

    list_metrics_shap['explainer'] = 'shap_1c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
    print("shap OK !!")
    
    # L2X
    e = L2X(model.predict, test_array_sampling)
    l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
    l2xvalues.shape

    # l2x's metrics
    list_metrics_l2x = {}
    l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

    list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
    list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
    list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
    list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                               samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
    list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e, L2X=True)
    list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
    list_metrics_l2x['explainer'] = 'l2x_1c'
    print("L2X OK !!")

    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])
df_metrics

1/1 [==============================] - 0s 44ms/step



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

18/18 [==============================] - 1s 41ms/step



  5%|████▏                                                                              | 1/20 [00:09<02:51,  9.04s/it]

18/18 [==============================] - 1s 45ms/step


...


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:58<00:13, 13.83s/it]

18/18 [==============================] - 1s 39ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:07<00:00, 24.39s/it]

L2X OK !!


,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,explainer
0,1.0,1.0,1.00,0.150924,4.145610,0.236745,0.631832,0.048846,lime_1c
0,1.0,1.0,0.95,0.147892,0.568901,0.232912,0.587377,0.453238,shap_1c
0,1.0,1.0,1.00,0.216381,2.378321,0.308456,0.615225,0.034186,l2x_1c
0,1.0,1.0,0.95,0.160015,0.218664,0.250107,0.744438,0.108766,lime_1c
0,1.0,1.0,0.95,0.157933,0.333230,0.251317,0.728482,0.434705,shap_1c
0,1.0,1.0,0.90,0.176160,38.070158,0.262740,0.783042,0.029785,l2x_1c
0,1.0,1.0,0.95,0.165309,0.751394,0.255895,0.771705,0.045492,lime_1c
0,1.0,1.0,0.95,0.165233,0.320011,0.255948,0.785162,0.403614,shap_1c
0,1.0,1.0,0.90,0.181828,13.867031,0.271893,0.787156,0.028032,l2x_1c


In [52]:
model_3c = model
model_3c.save('/sfds_result/model_3c')
df_metrics.to_csv('sfds_result/df_metric_sfds.csv')

INFO:tensorflow:Assets written to: /sfds_result/model_3c\assets


INFO:tensorflow:Assets written to: /sfds_result/model_3c\assets


In [54]:
seed = 0
tf.keras.utils.set_random_seed(seed)

## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [I. Mise en place des fonctions](#function)
    * [1. KernelSHAP](#KernelSHAP)
    * [2. L2X](#l2x)
    * [3. Génération des explications](#f_xai)
    * [4. Métriques d'évaluation](#metrique)
    * [5. Fonction pour la préparation des données et du modèle d'analyse](#dataprep)
* [II. Chargement des données](#data)
* [III. Expérimentation](#experiementation)
    * [1. Model à une couche](#model1layer)
    * [2. Model à 2 couches](#model2layer)
    * [3. Model à 3 couches](#model3layer)
    * [4. Model à 4 couches](#model4layer)
    * [5. Model à 5 couches](#model5layer)
* [IV. Evaluation des explication](#xai_metric)

### Model à 4 couches <a id="model4layer"></a>

In [55]:
# Modeling
if True:
    weights_file = str(alpha) + '4lstm_hyper_parameter_weights.h5'
    model = create_lstm_4layers(dropout, activation, weights_file, input_shape=input_shape)
    mse = []
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer='adam')
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

#             with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}
output.append(d)
model_4c = model
output

print(" Model evaluation for FD004 : ")
y_hat_train = model.predict(train_array)
evaluate(label_array, y_hat_train, 'train')

y_hat_test = model.predict(test_array)
evaluate(y_test, y_hat_test)
print(' ')
print(" Model evaluation for FD004 : ")

y_hat_test = model.predict(test_array)
evaluate(y_test['RUL'].clip(upper=upper), y_hat_test)
print(' ')
print("S-score for test : ",compute_s_score(y_test, y_hat_test))
print(' ')

8/8 [==============================] - 1s 30ms/step
 Model evaluation for FD004 : 
1611/1611 [==============================] - 51s 32ms/step
train set RMSE:9.015172958374023, R2:0.9218156234440671
8/8 [==============================] - 0s 33ms/step
test set RMSE:10.22274818629436, R2:0.9134238969173201
 
 Model evaluation for FD004 : 
8/8 [==============================] - 0s 33ms/step
test set RMSE:10.22274818629436, R2:0.9134238969173201
 
S-score for test :  515.5448483808688
 


In [87]:
RMSE.append(10.22)
Sscore.append(515.54)

In [90]:
df_perf = pd.DataFrame()
df_perf['RMSE'] = RMSE
df_perf["S-score"] = Sscore
df_perf.index = ["1_layer", "2_layers", "3_layers", "4_layers"]
df_perf.to_csv("sfds_result/df_perf.csv")

In [56]:
model_4c = model
model_4c.save('/sfds_result/model_4c')

INFO:tensorflow:Assets written to: /sfds_result/model_4c\assets


INFO:tensorflow:Assets written to: /sfds_result/model_4c\assets


In [57]:
if True:
    
    # LIME
    lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                       feature_names=remaining_sensors,
                                                       mode = 'regression',
                                                       )
    lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
    Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

    #Lime's metrics
    list_metrics_lime = {}
    list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
    list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
    list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
    list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                explainer = get_lime_explanation,
                                               samples=test_array_sampling,
                                                targets=label_array, e = model.predict)
    list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                               samples=test_array_sampling, e_x = model.predict)
    list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
    list_metrics_lime['explainer'] = 'lime_4c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])
    print("Lime OK!!")
    
    # SHAP
    e = KernelSHAP(model)
    shapvalues = get_explainations(test_array_sampling, e)
    shapvalues.shape

    list_metrics_shap = {}
    shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix
    
    list_metrics_shap['identity'] = identity(X_dist, shap_dist)
    list_metrics_shap['separability'] = separability(X_dist, shap_dist)
    list_metrics_shap['stability'] = stability(X_dist, shap_dist)
    list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                explainer = get_explainations,
                                               samples=test_array_sampling,
                                                targets=label_array, e = e)
    list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e)
    list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)

    list_metrics_shap['explainer'] = 'shap_4c'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
    print("shap OK !!")
    
    # L2X
    e = L2X(model.predict, test_array_sampling)
    l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
    l2xvalues.shape

    # l2x's metrics
    list_metrics_l2x = {}
    l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # Lime values explanation matrix

    list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
    list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
    list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
    list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                               samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
    list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                   samples=test_array_sampling, e_x=e, L2X=True)
    list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
    list_metrics_l2x['explainer'] = 'l2x_4c'
    print("L2X OK !!")

    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])
df_metrics

1/1 [==============================] - 0s 62ms/step



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

18/18 [==============================] - 1s 35ms/step



  5%|████▏                                                                              | 1/20 [00:08<02:38,  8.33s/it]

18/18 [==============================] - 1s 38ms/step


...


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [04:18<00:13, 13.55s/it]

18/18 [==============================] - 1s 36ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:21<00:00, 25.05s/it]

L2X OK !!


,identity,separability,stability,coherence,completness,congruence,selectivity,accumen,explainer
0,1.0,1.0,1.00,0.150924,4.145610,0.236745,0.631832,0.048846,lime_1c
0,1.0,1.0,0.95,0.147892,0.568901,0.232912,0.587377,0.453238,shap_1c
0,1.0,1.0,1.00,0.216381,2.378321,0.308456,0.615225,0.034186,l2x_1c
0,1.0,1.0,0.95,0.160015,0.218664,0.250107,0.744438,0.108766,lime_1c
0,1.0,1.0,0.95,0.157933,0.333230,0.251317,0.728482,0.434705,shap_1c
0,1.0,1.0,0.90,0.176160,38.070158,0.262740,0.783042,0.029785,l2x_1c
0,1.0,1.0,0.95,0.165309,0.751394,0.255895,0.771705,0.045492,lime_1c
0,1.0,1.0,0.95,0.165233,0.320011,0.255948,0.785162,0.403614,shap_1c
0,1.0,1.0,0.90,0.181828,13.867031,0.271893,0.787156,0.028032,l2x_1c
0,1.0,1.0,0.95,0.166296,32.262260,0.260225,0.764758,0.096995,lime_4c


In [58]:
df_metrics.to_csv('sfds_result/df_metric_sfds.csv')

In [83]:
df_output = pd.DataFrame(output[1])
df_output

,RMSE_val,RMSE_test,R2_test,Score_test,alpha,rul_piecewise,sequence_length
0,32.473881,35.244774,0.578884,10162.634392,0.5,100,40
1,9.544853,13.600943,0.578884,10162.634392,0.5,100,40
2,8.737921,9.888630,0.578884,10162.634392,0.5,100,40
